# Reading in and cleaning up VADIR  Data (then save to csv file)

Source:  http://www.p12.nysed.gov/irs/school_safety/school_safety_data_reporting.html

#### Still TO DO:
* clean up uppercase/lowercase issues (school name and county)
* fix column name duplicates and typos (alcohol and drug possession, sex offenses)
* check that schools are consistently assigned the same district name
* figure out a way to handle the data that doesn't have county info (search for school's subsequent year data?)
* get lattitude and longitude (and addresses???)
* re order columns in some meaningful way
* start computing tallies of incidents with and without weapons
* check datatypes for columns to make sure they make sense.

---
### Initial Set Up
To start we'll use the 2014-15 VARDIS data to get a list of column names and then use it to automate the "cleaning up" of the rest of the VARDIS files.

In [1]:
# Initial Imports
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
% matplotlib inline

In [3]:
# loading 2014-15 VADIR file
v2014_df = pd.read_excel("VADIR_2014.xls")
#v2014_df.head()

In [4]:
# Store row 1 to check that the other data sets have the same column names.
row1 = list(v2014_df.iloc[1])

In [5]:
# Now use row 1 to generate a list of corrected column names.
def clean_vars(df, row_num):
    """ 
    Helper function to clean up column names for VARDIS files.
    INPUT: dataframe, index of the row that contains the names
    OUTPUT: list of column names
    """
    names = list(df.iloc[row_num])
    for idx in range(len(names)):
        if str(names[idx]) == 'nan':
            names[idx] = names[idx - 1] + '_nw' # 'no weapon'
            names[idx - 1] += '_ww' # 'with weapon'
    return names

# take a look:
COLNAMES = clean_vars(v2014_df, 1)
COLNAMES

['County',
 'District',
 'School Name',
 'BEDS Code',
 'Grade Organization',
 'Need/Resource Category',
 'School Type',
 'Enrollment',
 'Homicide_ww',
 'Homicide_nw',
 'Forcible Sex Offenses_ww',
 'Forcible Sex Offenses_nw',
 'Other Sex Offenses_ww',
 'Other Sex Offenses_nw',
 'Robbery_ww',
 'Robbery_nw',
 'Assault With Serious Physical Injury_ww',
 'Assault With Serious Physical Injury_nw',
 'Arson',
 'Kidnapping_ww',
 'Kidnapping_nw',
 'Assault with Physical Injury_ww',
 'Assault with Physical Injury_nw',
 'Reckless Endangerment_ww',
 'Reckless Endangerment_nw',
 'Minor Altercations_ww',
 'Minor Altercations_nw',
 'Intimidation, Harassment, Menacing, or Bullying_ww',
 'Intimidation, Harassment, Menacing, or Bullying_nw',
 'Burglary_ww',
 'Burglary_nw',
 'Criminal Mischief_ww',
 'Criminal Mischief_nw',
 'Larceny, or Other Theft_ww',
 'Larceny, or Other Theft_nw',
 'Bomb Threat',
 'False Alarm',
 'Riot_ww',
 'Riot_nw',
 'Weapon Possession_ww',
 'Weapon Possession_nw',
 'Drug Possession

In [6]:
# reset column names, delete extra rows, add column for reporting year
v2014_df.columns = COLNAMES
v2014_df = v2014_df[3:]
v2014_df['School Year'] = ['2014'] * len(v2014_df)

# take a look
#v2014_df.head()

### Builing the VARDIS Dataframe  
Now that we have a clear sense of the column names and format of the data we'll build the actual dataset by reading in the rest of the files.

In [48]:
# start from the 2014-15 data
vadir_df = df = pd.read_excel('VADIR_2014.xls', names = COLNAMES, skiprows = 3)
vadir_df['School Year'] = ['2014'] * len(vadir_df)
print( 'Data from 2014-15 loaded. Total of {} rows.'.format(len(vadir_df)))

# import files for 2010-13 (they have same format as 2014 so this is quick)
files = ['VADIR_2013.xls', 'VADIR_2012.xls', 'VADIR_2011.xls', 'VADIR_2010.xls']
for f in files:
    df = pd.read_excel(f, names = COLNAMES, skiprows = 3)
    df['School Year'] = [f[-8:-4]] * len(df)
    vadir_df = pd.concat([vadir_df, df], ignore_index = True)
    print( '... data from {} appended. Added {} rows for a total of {}.'.format(f, len(df), len(vadir_df)))

Data from 2014-15 loaded. Total of 1805 rows.
... data from VADIR_2013.xls appended. Added 1792 rows for a total of 3597.
... data from VADIR_2012.xls appended. Added 1735 rows for a total of 5332.
... data from VADIR_2011.xls appended. Added 1693 rows for a total of 7025.
... data from VADIR_2010.xls appended. Added 1678 rows for a total of 8703.


In [49]:
# The remaining files have different column names so we'll do them by hand.
# ... starting with 2009
vadir_2009 = pd.read_excel('VADIR_2009.xls')
vadir_2009.columns = clean_vars(vadir_2009, 1)[:3] + ['District'] + clean_vars(vadir_2009, 1)[4:]
vadir_2009 = vadir_2009[3:]
#vadir_2009.head()
vadir_2009['School Year'] = [2009] * len(vadir_2009)
vadir_df = pd.concat([vadir_df, vadir_2009], ignore_index = True)
print( '... data from 2009-10 appended. Added {} rows for a total of {}.'.format(len(vadir_2009), len(vadir_df)))

... data from 2009-10 appended. Added 1531 rows for a total of 10234.


In [50]:
# ... next 2008
vadir_2008 = pd.read_excel('VADIR_2008.xls')
vadir_2008.columns = clean_vars(vadir_2008, 1)[:2] + ['District'] + clean_vars(vadir_2008, 1)[3:]
vadir_2008 = vadir_2008[3:]
#vadir_2008.head()
vadir_2008['School Year'] = [2008] * len(vadir_2008)
vadir_df = pd.concat([vadir_df, vadir_2008], ignore_index = True)
print( '... data from 2008-9 appended. Added {} rows for a total of {}.'.format(len(vadir_2008), len(vadir_df)))

... data from 2008-9 appended. Added 1545 rows for a total of 11779.


In [51]:
# ... next 2007
vadir_2007 = pd.read_excel('VADIR_2007.xls')
vadir_2007.columns = ['County', 'BEDS Code', 'District'] + clean_vars(vadir_2007, 0)[3:]
vadir_2007 = vadir_2007[2:]
#vadir_2007.head()
vadir_2007['School Year'] = [2007] * len(vadir_2007)
vadir_df = pd.concat([vadir_df, vadir_2007], ignore_index = True)
print( '... data from 2007-8 appended. Added {} rows for a total of {}.'.format(len(vadir_2007), len(vadir_df)))

... data from 2007-8 appended. Added 1500 rows for a total of 13279.


In [52]:
# ... next 2006
vadir_2006 = pd.read_excel('VADIR_2006.xls')
vadir_2006.columns = ['County', 'District', 'School Name', 'BEDS Code'] + clean_vars(vadir_2006, 0)[4:]
vadir_2006 = vadir_2006[2:]
#vadir_2006.head()
vadir_2006['School Year'] = [2006] * len(vadir_2006)
vadir_df = pd.concat([vadir_df, vadir_2006], ignore_index = True)
print( '... data from 2006-7 appended. Added {} rows for a total of {}.'.format(len(vadir_2006), len(vadir_df)))

... data from 2006-7 appended. Added 1455 rows for a total of 14734.


### A look at the resulting DataFrame (and then saving it to csv)

In [53]:
# Quick Check  - counts by year
vadir_df['School Year'].value_counts()

2014    1805
2013    1792
2012    1735
2011    1693
2010    1678
2008    1545
2009    1531
2007    1500
2006    1455
Name: School Year, dtype: int64

In [54]:
# A glance at the final data frame:
vadir_df.head()

,Alcohol Possesion,Alcohol Possession,Arson,Assault With Serious Physical Injury_nw,Assault With Serious Physical Injury_ww,Assault with Physical Injury_nw,Assault with Physical Injury_ww,BEDS Code,Bomb Threat,Burglary_nw,...,Riot_ww,Robbery_nw,Robbery_ww,School Name,School Type,School Year,Use Possession or Sale of Alcohol,Use Possession or Sale of Drugs,Weapon Possession_nw,Weapon Possession_ww
0,NaN,0,0,0,0,0,0,320700860957,0,0,...,0,0,0,Academic Leadership Charter School,Charter,2014,NaN,NaN,0,0
1,NaN,0,0,0,0,0,0,320700861062,0,0,...,0,0,0,American Dream Charter School,Charter,2014,NaN,NaN,0,0
2,NaN,0,0,0,0,0,0,320700861014,0,0,...,0,0,0,Brilla College Preparatory Charter School,Charter,2014,NaN,NaN,0,0
3,NaN,0,0,0,0,0,0,320900860913,0,0,...,0,0,0,Bronx Academy Of Promise Charter School,Charter,2014,NaN,NaN,0,0
4,NaN,0,0,0,0,0,0,321100860855,0,0,...,0,0,0,Bronx Charter School For Better Learning,Charter,2014,NaN,NaN,0,0


In [56]:
list(vadir_df.columns)

['Alcohol Possesion',
 'Alcohol Possession',
 'Arson',
 'Assault With Serious Physical Injury_nw',
 'Assault With Serious Physical Injury_ww',
 'Assault with Physical Injury_nw',
 'Assault with Physical Injury_ww',
 'BEDS Code',
 'Bomb Threat',
 'Burglary_nw',
 'Burglary_ww',
 'County',
 'Criminal Mischief_nw',
 'Criminal Mischief_ww',
 'District',
 'Drug Possesion',
 'Drug Possession',
 'Enrollment',
 'False Alarm',
 'Forcible Sex Offenses_nw',
 'Forcible Sex Offenses_ww',
 'Grade Organization',
 'Homicide_nw',
 'Homicide_ww',
 'Intimidation, Harassment, Menacing, or Bullying_nw',
 'Intimidation, Harassment, Menacing, or Bullying_ww',
 'Kidnapping_nw',
 'Kidnapping_ww',
 'Larceny, or Other Theft_nw',
 'Larceny, or Other Theft_ww',
 'Minor Altercations_nw',
 'Minor Altercations_ww',
 'Need/Resource Category',
 'Other Disruptive',
 'Other Disruptive Incidents',
 'Other Sex Offenses_nw',
 'Other Sex Offenses_ww',
 'Other Sex offenses_nw',
 'Other Sex offenses_ww',
 'Reckless Endangerment

In [57]:
# Saving the data to a new file
vadir_df.to_csv("VADIR_incidents_2006-14.csv")

In [61]:
#checking that it saved properly
pd.read_csv("VADIR_incidents_2006-14.csv", dtype = object).head()
# looks like the BEDS code is wrongly categorized as a float... 
# I'll temporarily fix that by importing it as an object but we should check the rest of the types.

,Unnamed: 0,Alcohol Possesion,Alcohol Possession,Arson,Assault With Serious Physical Injury_nw,Assault With Serious Physical Injury_ww,Assault with Physical Injury_nw,Assault with Physical Injury_ww,BEDS Code,Bomb Threat,...,Riot_ww,Robbery_nw,Robbery_ww,School Name,School Type,School Year,Use Possession or Sale of Alcohol,Use Possession or Sale of Drugs,Weapon Possession_nw,Weapon Possession_ww
0,0,NaN,0,0,0,0,0,0,320700860957,0,...,0,0,0,Academic Leadership Charter School,Charter,2014,NaN,NaN,0,0
1,1,NaN,0,0,0,0,0,0,320700861062,0,...,0,0,0,American Dream Charter School,Charter,2014,NaN,NaN,0,0
2,2,NaN,0,0,0,0,0,0,320700861014,0,...,0,0,0,Brilla College Preparatory Charter School,Charter,2014,NaN,NaN,0,0
3,3,NaN,0,0,0,0,0,0,320900860913,0,...,0,0,0,Bronx Academy Of Promise Charter School,Charter,2014,NaN,NaN,0,0
4,4,NaN,0,0,0,0,0,0,321100860855,0,...,0,0,0,Bronx Charter School For Better Learning,Charter,2014,NaN,NaN,0,0
